In [50]:
import numpy as np
import model
from importlib import reload
reload(model)


<module 'model' from 'C:\\Users\\Paul\\Documents\\Keras-progressive_growing_of_gans\\Progressive growing of GANs\\model.py'>

In [51]:
G = model.Generator(use_wscale=False)

In [7]:
import keras
import pydotplus
from keras.utils.vis_utils import model_to_dot
import pydot
from keras.utils import plot_model
keras.utils.vis_utils.pydot = pydot
plot_model(G, to_file='model.png')

In [52]:
#def insert_layer_nonseq(model, layer_regex, insert_layer_factory, insert_layer_name=None, position='after'):
model = G

# Auxiliary dictionary to describe the network graph
network_dict = {'input_layers_of': {}, 'new_output_tensor_of': {}}

# Set the input layers of each layer
for layer in model.layers:
    for node in layer.outbound_nodes:
        layer_name = node.outbound_layer.name
        if layer_name not in network_dict['input_layers_of']:
            network_dict['input_layers_of'].update(
                    {layer_name: [layer.name]})
        else:
            network_dict['input_layers_of'][layer_name].append(layer.name)

# Set the output tensor of the input layer
network_dict['new_output_tensor_of'].update(
        {model.layers[0].name: model.input})
print(network_dict)

{'input_layers_of': {'Gnorm': ['Glatents'], 'G1nb': ['Gnorm'], 'G1aPad': ['G1nb'], 'G1a': ['G1aPad'], 'G1aBN': ['G1a'], 'G1aPN': ['G1aBN'], 'G1bPad': ['G1aPN'], 'G1b': ['G1bPad'], 'G1bBN': ['G1b'], 'G1bPN': ['G1bBN'], 'G2up': ['G1bPN'], 'Glod3NIN': ['G1bPN'], 'G2aPad': ['G2up'], 'G2a': ['G2aPad'], 'G2aBN': ['G2a'], 'G2aPN': ['G2aBN'], 'G2bPad': ['G2aPN'], 'G2b': ['G2bPad'], 'G2bBN': ['G2b'], 'G2bPN': ['G2bBN'], 'G3up': ['G2bPN'], 'Glod2NIN': ['G2bPN'], 'G3aPad': ['G3up'], 'G3a': ['G3aPad'], 'G3aBN': ['G3a'], 'G3aPN': ['G3aBN'], 'G3bPad': ['G3aPN'], 'G3b': ['G3bPad'], 'G3bBN': ['G3b'], 'G3bPN': ['G3bBN'], 'G4up': ['G3bPN'], 'Glod1NIN': ['G3bPN'], 'G4aPad': ['G4up'], 'G4a': ['G4aPad'], 'G4aBN': ['G4a'], 'G4aPN': ['G4aBN'], 'G4bPad': ['G4aPN'], 'G4b': ['G4bPad'], 'G4bBN': ['G4b'], 'G4bPN': ['G4bBN'], 'Glod0NIN': ['G4bPN'], 'actv_resize_layer_33': ['Glod0NIN'], 'Glod': ['Glod0NIN', 'Glod1NIN', 'Glod2NIN', 'Glod3NIN'], 'actv_resize_layer_34': ['Glod1NIN'], 'actv_resize_layer_35': ['Glod2NIN

In [53]:
import re
from keras.models import Model

# source : https://stackoverflow.com/questions/49492255/how-to-replace-or-insert-intermediate-layer-in-keras-model

# Iterate over all layers after the input
for layer in model.layers[1:]:

    # Determine input tensors
    layer_input = [network_dict['new_output_tensor_of'][layer_aux] 
            for layer_aux in network_dict['input_layers_of'][layer.name]]
    if len(layer_input) == 1:
        layer_input = layer_input[0]

    # Insert layer if name matches the regular expression
    if 'BN' in layer.name:
        # replace layer
        x = layer_input

        new_layer = BatchNormalization(name=layer.name + '_new')
        
        x = new_layer(x)
        print('Layer {} inserted after layer {}'.format(new_layer.name, layer.name))

    else:
        x = layer(layer_input)

    # Set new output tensor (the original one, or the one of the inserted
    # layer)
    network_dict['new_output_tensor_of'].update({layer.name: x})
    
G_twin = Model(inputs=model.inputs, outputs=x)
#return Model(inputs=model.inputs, outputs=x)

Layer G1aBN_new inserted after layer G1aBN
Layer G1bBN_new inserted after layer G1bBN
Layer G2aBN_new inserted after layer G2aBN
Layer G2bBN_new inserted after layer G2bBN
Layer G3aBN_new inserted after layer G3aBN
Layer G3bBN_new inserted after layer G3bBN
Layer G4aBN_new inserted after layer G4aBN
Layer G4bBN_new inserted after layer G4bBN
